In [1]:
#reading in data
import pandas as pd 
import numpy as np 
import ast
data = pd.read_csv('movies_metadata_sample.csv', converters= {'genres' : ast.literal_eval})
df = data.explode('genres')

In [2]:
#create a function that searches for missing values
def missing(x):
    return sum(x.isnull())

#applying per column
print ("Missing values per column:")
print (df.apply(missing, axis =0))

#applying per row
print ("\nMissing values per row:")
print (df.apply(missing, axis =1).head())

Missing values per column:
adult                        0
belongs_to_collection     9871
budget                       0
genres                     106
homepage                 11300
id                           0
imdb_id                      0
original_language            0
original_title               0
overview                    31
popularity                   0
poster_path                 27
production_companies         0
production_countries         0
release_date                 4
revenue                      0
runtime                      6
spoken_languages             0
status                       8
tagline                   2618
title                        0
video                        0
vote_average                 0
vote_count                   0
dtype: int64

Missing values per row:
0    3
0    3
1    2
1    2
2    2
dtype: int64


In [3]:
#Creating a column called year and filling it with year of movie
df['year'] = pd.DatetimeIndex(df['release_date']).year

#finding missing values for year
df['year'] = df['year'].replace(np.nan, 0)
missingYear = df.loc[(df['year'] == 0), ['id', 'title', 'genres', 'year']]

#replacing values for missing years
df.loc[df.id == 365371, ['year', 'genres']] = 1995, "[{'id': 99, 'name': 'Documentary'}]"
df.loc[df.id == 215107, ['year', 'genres']] = 1992, "[{'id': 35, 'name': 'Comedy'}]"
df.loc[df.id == 94214, 'year'] = 2000 
df.loc[df.id == 207731, ['year', 'genres']] = 2000, "[{'id': 18, 'name': 'Drama'}]"


In [5]:
#converting genres to a string from object
data2 = df['genres'].astype(str)

#splitting genres id and name into new columns called gid, genre
data3 = pd.DataFrame(data2.str.split(',').tolist(), columns = ['gid', 'genre'])

#removing all parts of genre column except the genre name
data3['genre'] = data3['genre'].map(lambda x: x and x.lstrip("'name': '").rstrip("'}"))


In [6]:
#resetting df index to get rid of duplicate values
df.reset_index(inplace= True)
#merging the year onto the updated datafram
data3['year'] = df['year'].astype(int)

In [7]:
#adjusting the year
def fix_year(x):
    if x > 2017:
        year = x - 100
    else:
        year = x
    return year

data3['year'] = data3['year'].apply(fix_year)            

In [8]:
#grouping the year and genre together with the count
final_data= data3.groupby(['year', 'genre']).size().to_frame('count').reset_index()


In [9]:
#exporting to csv file
final_data.to_csv('Final_Output', index= False)